In [1]:
#from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd

In [2]:
training_data = "/Users/geanmotta/Desktop/Estudos em investimentos/Dados de emprestimos/archive/lc_2016_2017.csv"
loans = pd.read_csv(training_data, low_memory = False)
#loans = loans[loans["purpose"] == "renewable_energy"]

validation_data = "/Users/geanmotta/Desktop/Estudos em investimentos/Dados de emprestimos/archive/lc_loan.csv"
new_loans = pd.read_csv(validation_data, low_memory = False)

In [3]:
loans["term"] = loans["term"].str.replace(" months", "")
loans.term = loans["term"].astype(int)

new_loans["term"] = new_loans["term"].str.replace(" months", "")
new_loans.term = new_loans["term"].astype(int)

In [4]:
criteria = ["loan_amnt", "term", "installment"] #,grade

# #Remove grade column above and comment this block to remove credit ratings from the criteria
# credit_ratings = loans.grade
# ratings_encoder = LabelEncoder()
# X_grades = (ratings_encoder.fit_transform(credit_ratings))
# loans.loc[:, "grade"] = X_grades.copy()


train_X = loans[criteria]
train_y = loans["int_rate"]


#You can simply use one of the files, or use past data as the training set and present data for evaluation
val_X = new_loans[criteria]
val_y = new_loans["int_rate"]

In [5]:
#To evaluate MAE
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0) #DecisionTreeRegressor
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

#For the max_leaf_nodes argument in the analysis model
my_branches = [5, 50, 500]

#Function to find smalles MAE
def shortest_mae(nodes):
    tested_maes = []
    for i in nodes:
        tested_maes.append(get_mae(i, train_X, val_X, train_y, val_y))
    return nodes[tested_maes.index(min(tested_maes))]


analysis_model = RandomForestRegressor(max_leaf_nodes = shortest_mae(my_branches), random_state = 0) #(random_state = 1)

In [6]:
analysis_model.fit(train_X, train_y)

predicted_rates = analysis_model.predict(val_X)

In [7]:
mean_absolute_error(predicted_rates, val_y)

0.6455564330756026